In [1]:
from nonTracking_02 import *

#=============================================================================
# Parameters
#=============================================================================
# Initialize seed of PRNG:
seed(129041982)
#-----------------------------------------------------------------------------
# Physical parameters:
#-----------------------------------------------------------------------------
S=10. # Surface of box [mu^2]
L=sqrt(S) # Linear size of box [mu]
rho=1. # Particle density [1/mu^2]
avg_N=int(rho*L**2) # Average number of particles in each frame
D=10. # Diffusion coefficient [mu^2/s]
dt=0.01 # Time lapse between images [s]
p_off=0.0 # Probability for each particle to disappear per frame
mu_off=avg_N*p_off # Average number of particles disappearing per frame
mu_on=mu_off # Average number of particles appearing per frame (set equal to mu_off)
nF=10 # Number of time lapses recorded (= number of frames minus one)
ensembleSize=1 # Size of Monte Carlo ensemble
tries=1 # Number of 
nL=0.0001
#-----------------------------------------------------------------------------
# Fit parameters:
#-----------------------------------------------------------------------------
Da=0.1*D
Db=10.*D


In [6]:
t_tot=time()
print "Parameter values: L =",L, "; rho =",rho, "; D =",D, "; dt =",dt, "; p_off =",p_off, "; mu_on =", mu_on,\
        "; number of frames:",nF

# Generate trajectories:
Ns_true, drs_true, Ns, drs_ij = simulate_BM(L, rho, D, dt, p_off, mu_on, nF, avg_N)
print "\Average number of particles per frame: "+str(np.average(Ns))
print "Average number of true links per frame: "+str(np.average(Ns_true))

print "\nFit using:"
# True assignment:
t_true=time()
sol_true=np.sum([np.average(dr_**2)*len(dr_)/(2.*dt) for dr_ in drs_true])/np.sum([len(dr_) for dr_ in drs_true])
print "  --- true matching: ---"
print "    D_est =", sol_true, "(time to fit: "+str(time()-t_true)+"s )" 

# Sum-product BP:
t_BP=time()
sol_BP=optim.minimize_scalar(marginal_minusLogLikelihood_multiFrame,bounds=[Da,Db],\
                                 args=(dt,drs_ij,L,mu_off,mu_on,Ns),method='bounded')
print "  --- non-tracking (BP): ---"
print "    D_est =", sol_BP.x, "(time to fit: "+str(time()-t_BP)+"s )" 

# MPA:
t_MPA=time()
sol_MP=optim.minimize_scalar(MPA_minusLogLikelihood_multiFrame,bounds=[Da,Db],\
                                 args=(dt,drs_ij,L,mu_off,mu_on,Ns),method='bounded')
print " --- most probable assignment: ---"
print "    D_est =", sol_MP.x, "(time to fit: "+str(time()-t_MPA)+"s )" 


Parameter values: L = 3.16227766017 ; rho = 1.0 ; D = 10.0 ; dt = 0.01 ; p_off = 0.0 ; mu_on = 0.0 ; number of frames: 10
\Average number of particles per frame: 10.0
Average number of true links per frame: 10.0

Fit using:
  --- true matching: ---
    D_est = 9.74156643027 (time to fit: 0.000655889511108s )
  --- non-tracking (BP): ---
    D_est = 9.08685720403 (time to fit: 7.93795990944s )
 --- most probable assignment: ---
    D_est = 7.98914677008 (time to fit: 0.0948519706726s )
